# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#!pip install gmaps

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
city_weather_df = pd.read_csv("city_weather.csv")

city_weather_df[['Humidity']] = city_weather_df[['Humidity']].astype(float)
city_weather_df[['Lat', 'Lng']] = city_weather_df[['Lat', 'Lng']].astype(float)

city_weather_df = city_weather_df.dropna()

city_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,0,Novichikha,52.2040,81.3877,8.44,overcast clouds,85.0,100,15.86,RU
1,1,Banjar,-8.1900,114.9675,77.99,broken clouds,89.0,78,0.94,ID
2,2,Paita,-5.0892,-81.1144,75.61,few clouds,66.0,11,4.18,PE
3,3,Ribeira Grande,38.5167,-28.7000,64.80,scattered clouds,77.0,40,20.71,PT
4,4,Rikitea,-23.1203,-134.9692,79.00,light rain,78.0,9,16.84,PF
...,...,...,...,...,...,...,...,...,...,...
570,570,Mahajanga,-15.7167,46.3167,78.82,thunderstorm with light rain,94.0,75,19.57,MG
571,571,Boda,4.3189,17.4695,97.99,overcast clouds,11.0,89,8.23,CF
572,572,La Paz,-16.5000,-68.1500,57.18,broken clouds,43.0,75,9.22,BO
573,573,Lhokseumawe,5.1801,97.1507,76.59,overcast clouds,86.0,100,4.29,ID


In [13]:
city_weather_df.dtypes

Unnamed: 0               int64
City                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Current Description     object
Humidity               float64
Cloudiness               int64
Wind Speed             float64
Country                 object
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
gmaps.configure(api_key=g_key)

In [15]:
locations = city_weather_df[['Lat', 'Lng']]
humidity = city_weather_df['Humidity']

In [16]:
city_weather_df.dtypes

Unnamed: 0               int64
City                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Current Description     object
Humidity               float64
Cloudiness               int64
Wind Speed             float64
Country                 object
dtype: object

In [19]:
fig = gmaps.figure(map_type="HYBRID")
heatmap_layer = gmaps.heatmap_layer(locations, humidity)
heatmap_layer.max_intensity = 75
heatmap_layer.point_radius = 3


fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [26]:
cycling_weather = city_weather_df

#Using the .loc function to find the ideal weather conditions for a cycling vacation (not too hot, cold, or cloudy and wind less than 10 mph)
cycling_weather = cycling_weather.loc[(cycling_weather["Max Temp"] <= 78) & (cycling_weather["Max Temp"] >= 50)]
cycling_weather = cycling_weather.loc[cycling_weather["Cloudiness"] <= 50]
cycling_weather = cycling_weather.loc[cycling_weather["Wind Speed"] <= 10]

cycling_weather = cycling_weather.dropna()

cycling_weather['Hotel Name'] = ""
cycling_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
2,2,Paita,-5.0892,-81.1144,75.61,few clouds,66.0,11,4.18,PE,
15,15,Puerto Ayora,-0.7393,-90.3518,76.96,scattered clouds,94.0,35,3.00,EC,
27,27,Hobart,-42.8794,147.3294,61.05,few clouds,91.0,21,4.61,AU,
31,31,Te Horo,-40.8084,175.1233,55.80,few clouds,87.0,24,4.29,NZ,
40,40,Pokhara,28.2333,83.9833,57.42,clear sky,23.0,0,6.87,NP,
...,...,...,...,...,...,...,...,...,...,...,...
529,529,Kupang,-10.1667,123.5833,75.78,few clouds,94.0,20,2.30,ID,
535,535,Jalu,29.0331,21.5482,66.69,clear sky,19.0,0,6.78,LY,
538,538,Iquique,-20.2208,-70.1431,77.04,clear sky,70.0,0,5.82,CL,
547,547,Frontera,26.9333,-101.4500,53.35,clear sky,58.0,0,1.01,MX,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
hotel_df = cycling_weather

base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"

params = {"radius": 5000,
          "type": "lodging",
         "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f"{lat}, {lng}"
    
    response = requests.get(base_url, params=params)
    
    response = response.json()
    
    #try and except for hotels in the cities on my cycling weather df
    
    try:
        print(f"Hotel Name is {results[0]['Hotel Name']} City No. {index}: {row['City']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['Hotel Name']
        print("----------------------------------------------")
              
    except:
        print("City not found. Skipping...")

City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skip

In [ ]:
# suspect there won't be anything in the df if I try to run and display it because the above won't work properly.
hotel_df = hotel_df.dropna()
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer on top of heat map
hotel_markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker)
# Display figure
fig